In [ ]:
## where to write temporary files 
TMP_PREFIX = "tmp/"

## csv file to find the annotations
ANNOTATION_FILE = "toy_data/test_annotation.csv"



mains steps: 
1. check overlap between parts
2. check overlap between cutout and parts extremities: push part extremities 
3. from cut out to sub parts
4. generate split and merge commands
5. apply commands






In [ ]:
from collections import defaultdict
import os 
import pandas as pd
import numpy as np

from utils import *

In [ ]:
df_annot = pd.read_csv(ANNOTATION_FILE)
df_annot['start_second'] = df_annot.start.apply(time_to_seconds) ## convert time to seconds
df_annot['stop_second'] = df_annot.stop.apply(time_to_seconds)   ## convert time to seconds


## check that all elements start before they stop:
PB = df_annot.loc[ df_annot.stop_second < df_annot.start_second , :]
if PB.shape[0]>0:
    print("PROBLEM: some elements stop before they start! please fix this")
    print( PB )
    
df_annot

In [ ]:
## separate parts and cut out
df_annot_parts = df_annot.loc[ df_annot.destination != "OUT" , :].copy().reset_index(drop=True)
df_annot_OUT = df_annot.loc[ df_annot.destination == "OUT" , :].copy().reset_index(drop=True)


In [ ]:
## check if some parts overlap :
check_overlap( df_annot_parts )


In [ ]:
## merge overlapping cutouts 
df_annot_OUT = merge_overlapping_elements(df_annot_OUT)

##sort cutout by video and start!!
df_annot_OUT.sort_values( by = ['source' , 'start_second']  , inplace=True)


In [ ]:
# applying cutouts :
#  * overlaps is across start or stop of part -> move part start or stop
#  * otherwise part is split in 2
df_subparts = associate_parts_and_cutout(df_annot_parts,df_annot_OUT)
df_subparts

In [ ]:
cmds = create_commands( df_subparts , prefix = TMP_PREFIX )
cmds

In [ ]:
# apply commands - this may take a long while 
for part , commands in cmds.items():
    print('creating',part)
    for c in commands:
        os.system(c)